# 回顾自动化机器学习解释

随着机器学习越来越普遍，模型做出的预测对我们社会的许多方面产生了更大的影响。例如，机器学习模型在银行如何决定发放贷款或医生如何确定治疗优先顺序方面成为了一个愈发重要的因素。解释和说明模型的功能变得越来越重要，以便可以说明和证明机器学习模型做出预测的基本原理，并且可以确定模型中任何无意的偏差。

使用自动化机器学习来训练模型时，可以选择生成特征重要性的解释，以量化每个特征（包括通过预处理转换生成的工程特征）对标签预测的影响程度。在本实验中，你将探索由自动化机器学习试验生成的解释。


## 连接到工作区

你首先需要使用 Azure ML SDK 连接到工作区。

> **注意**： 如果 Azure 订阅的身份验证会话在你完成上一练习后已过期，系统将提示你重新进行身份验证。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 从保存的配置文件加载工作区
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 运行自动化机器学习试验

在本实验中，为节省时间，你将在本地计算机上运行自动化机器学习试验，只有三次迭代。

请注意，将 **model_explainability** 配置选项设为 **True**。

In [ ]:
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

# 加载数据
train_data = pd.read_csv('data/diabetes.csv')

# 配置自动化 ML
automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target='local',
                             training_data = train_data,
                             n_cross_validations = 2,
                             label_column_name='Diabetic',
                             iterations=3,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=3,
                             featurization='off',
                             model_explainability=True # 生成特征重要性！
                             )

# 运行自动化 ML 试验
print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'diabetes_automl')
automl_run = automl_experiment.submit(automl_config)
automl_run.wait_for_completion(show_output=True)
RunDetails(automl_run).show()

## 查看特征重要性

完成试验后，在上面的小部件中，单击生成最佳结果的运行以查看其详细信息。然后滚动到可视化效果底部以查看相对特征重要性。

还可以使用 **ExplanationClient** 类查看该试验生成的最佳模型的特征重要性：

In [ ]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient

# 获取最佳模型（输出中的第 2 项）
best_run, fitted_model = automl_run.get_output()

# 获取特征解释
client = ExplanationClient.from_run(best_run)
engineered_explanations = client.download_model_explanation()
feature_importances = engineered_explanations.get_feature_importance_dict()

# 整体特征重要性
print('Feature\tImportance')
for key, value in feature_importances.items():
    print(key, '\t', value)

## 在 Azure 机器学习工作室中查看模型解释

完成试验运行后，单击小组件中的链接以在 Azure 机器学习工作室中查看运行，然后查看 **解释** 选项卡。然后：

1. 选择由自动化机器学习运行创建的解释器。
2. 查看 **全局重要性** 图表，其中显示整体全局特征重要性。
3. 查看 **摘要重要性** 图表，其中显示 *分簇散点图*、 *小提琴图* 或 *盒须图* 中测试数据的每个数据点。
4. 选择单个点以查看所选数据点的单个预测的 **本地特征重要性**。

## 查看自动工程特征的重要性

自动化机器学习包括尝试预处理数据的选项，这通常执行 *特征工程* 以创建新的派生特征，模型是根据这些特征进行训练的。现在启用该选项并重新运行自动化机器学习试验。

In [ ]:
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

# 加载数据
train_data = pd.read_csv('data/diabetes.csv')

# 配置自动化 ML
automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target='local',
                             training_data = train_data,
                             n_cross_validations = 2,
                             label_column_name='Diabetic',
                             iterations=3,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=3,
                             featurization='auto', # 启用特征工程
                             model_explainability=True # 生成特征重要性
                             )

# 运行自动化 ML 试验
print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'diabetes_automl')
automl_run = automl_experiment.submit(automl_config)
automl_run.wait_for_completion(show_output=True)
RunDetails(automl_run).show()

特征化是通过使用 [Scikit-Learn 转换管道](https://scikit-learn.org/stable/modules/compose.html#combining-estimators)（不要与 Azure 机器学习管道混淆！）实现的。所生成的模型包括在推理之前转换数据的步骤。

运行以下代码以查看模型管道中的步骤以及工程特征的重要性。

In [ ]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient

# 获取最佳模型（输出中的第 2 项）
best_run, fitted_model = automl_run.get_output()

print('Model pipeline steps:')
for step in fitted_model.named_steps:
    print('-',step)
    
# 获取特征解释
client = ExplanationClient.from_run(best_run)
engineered_explanations = client.download_model_explanation(raw=False)
feature_importances = engineered_explanations.get_feature_importance_dict()

# 整体特征重要性
print('\nFeature\tImportance')
for key, value in feature_importances.items():
    print(key, '\t', value)

> **更多信息**： 有关自动化机器学习的详细信息，请参阅 [Azure ML 文档](https://docs.microsoft.com/azure/machine-learning/how-to-configure-auto-train)。